In [1]:
import numpy as np
import pandas as pd
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import SparkConf,SparkContext

sc = SparkContext("local", "model training")

In [2]:
train  = sc.textFile('E:/CMPE255/train.csv')
test  = sc.textFile('E:/CMPE255/test.csv')

header = train.first()
train = train.filter(lambda x: x != header)

header = test.first()
test = test.filter(lambda x: x != header)
ratings = train.map(lambda l: l.split(',')).map(lambda l:Rating(int(l[0]),int(l[1]),float(l[2])))

In [3]:
# ratings = train.map(lambda l: l.split(',')).map(lambda l:Rating(int(l[0]),int(l[1]),float(l[2])))
rank_list = [2,4,6,8,10,15,20,30,40,50]
numItera_list = [5,10,20,30,40]

valData = train.map(lambda l: l.split(',')).map(lambda l: (l[0],l[1]))
testData = test.map(lambda l: l.split(',')).map(lambda l: (l[0],l[1]))

rmse_train = [[0]*len(numItera_list) for _ in range(len(rank_list))]
rmse_test = [[0]*len(numItera_list) for _ in range(len(rank_list))]

In [4]:
for rank_id,rank in enumerate(rank_list):
    for numItera_id,numItera in enumerate(numItera_list):
        model = ALS.train(ratings,rank,numItera)
        predictionVal = model.predictAll(valData).map(lambda l:((l[0],l[1]),l[2]))
        r_p = train.map(lambda l: l.split(',')).map(lambda l: ((int(l[0]),int(l[1])),float(l[2]))).join(predictionVal)
        MSE = r_p.map(lambda l:(l[1][0]-l[1][1])**2).mean()
        print("MES value: ",str(MSE))
        rmse_train[rank_id][numItera_id] = float(MSE)
        
        prediction = model.predictAll(testData).map(lambda l:((l[0],l[1]),l[2]))
        r_p = test.map(lambda l: l.split(',')).map(lambda l: ((int(l[0]),int(l[1])),float(l[2]))).join(prediction)
        MSE = r_p.map(lambda l:(l[1][0]-l[1][1])**2).mean()
        rmse_test[rank_id][numItera_id] = float(MSE)
        

MES value:  0.7211634437265813
MES value:  0.6869385439034954


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 18 in stage 678.0 failed 1 times, most recent failure: Lost task 18.0 in stage 678.0 (TID 1268, localhost, executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:111)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:155)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:122)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.io.FilterOutputStream.write(FilterOutputStream.java:97)
	at org.apache.spark.api.python.PythonRDD$.org$apache$spark$api$python$PythonRDD$$write$1(PythonRDD.scala:212)
	at org.apache.spark.api.python.PythonRDD$$anonfun$writeIteratorToStream$1.apply(PythonRDD.scala:224)
	at org

In [ ]:
err_train = pd.Dataframe(rmse_train,column=numItera_list,index=rank_list)
err_test = pd.Dataframe(rmse_test,column=numItera_list,index=rank_list)

err_train.to_csv('err_train.csv')
err_test.to_csv('err_test.csv')

In [0]:
# valData = train.map(lambda l: l.split(',')).map(lambda l: (l[0],l[1]))
# predictionVal = model.predictAll(valData).map(lambda l:((l[0],l[1]),l[2]))
# r_p = train.map(lambda l: l.split(',')).map(lambda l: ((int(l[0]),int(l[1])),float(l[2]))).join(predictionVal)
# MSE = r_p.map(lambda l:(l[1][0]-l[1][1])**2).mean()
# print("MES value: ",str(MSE))


MES value:  0.5095856010332751


In [0]:
# testData = test.map(lambda l: l.split(',')).map(lambda l: (l[0],l[1]))
# prediction = model.predictAll(testData).map(lambda l:((l[0],l[1]),l[2]))
# r_p = test.map(lambda l: l.split(',')).map(lambda l: ((int(l[0]),int(l[1])),float(l[2]))).join(prediction)
# MSE = r_p.map(lambda l:(l[1][0]-l[1][1])**2).mean()
# print("MES value: ",str(MSE))

MES value:  0.6277331427332887
